In [ ]:
!pip install flask-ngrok


In [ ]:
from google.colab import drive 
drive.mount('/content/drive')
!ls "/content/drive/MyDrive"

Mounted at /content/drive
'Colab Notebooks'   predictive_maintenance.csv	 Untitled


In [ ]:
cd '/content/drive/MyDrive/Colab Notebooks/Data/Flask-deploy-model'

/content/drive/MyDrive/Colab Notebooks/Data/Flask-deploy-model


In [ ]:
!ngrok authtoken '26gxiHBGFtcAzMGXavNu81BI1IG_4aRYdXCSk51MvzAh7v558'

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


# MODEL BUILDING AND DUMP USING PICKLE

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
import pickle


data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data/Flask-deploy-model/insurance.csv")
le = LabelEncoder()
le.fit(data['sex'])
data['Sex'] = le.transform(data['sex'])
le.fit(data['smoker'])
data['Smoker'] = le.transform(data['smoker'])
le.fit(data['region'])
data['Region'] = le.transform(data['region'])
#independent and dependent columns
x = data[["age", "bmi", "children", "Sex", "Smoker", "Region"]]
y = data['expenses']
#split in train and test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)
#model training
linreg = LinearRegression()
linreg.fit(x_train, y_train)
#model testing
predictions = linreg.predict(x_test)
linreg.score(x_test,y_test)
#save the model
file = open("/content/drive/MyDrive/Colab Notebooks/Data/Flask-deploy-model/expense_model.pkl", 'wb')
pickle.dump(linreg, file)

# READ MODEL FROM PICKLE AND RUN APP

In [ ]:
from flask import Flask, request, render_template
import pickle
import numpy as np
from flask_ngrok import run_with_ngrok


app = Flask(__name__ , template_folder='/content/drive/MyDrive/Colab Notebooks/Data/Flask-deploy-model/templates')
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/")
def hello():
    #return "Hello World!"
    return render_template("index.html")
@app.route("/predict", methods=['GET','POST'])
def predict():
   if request.method == 'POST':
        #access the data from form
        ## Age
        age = int(request.form["age"])
        bmi = int(request.form["bmi"])
        children = int(request.form["children"])
        Sex = int(request.form["Sex"])
        Smoker = int(request.form["Smoker"])
        Region = int(request.form["Region"])
        #get prediction
        input_cols = [[age, bmi, children, Sex, Smoker, Region]]
        prediction = model.predict(input_cols)
        output = round(prediction[0], 2)
        return render_template("index.html", prediction_text='Your predicted annual Healthcare Expense is $ {}'.format(output))
if __name__ == '__main__':
    app.run()  # If address is in use, may need to terminate other sessions:
               # Runtime > Manage Sessions > Terminate Other Sessions

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://869c-35-234-50-53.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [23/Mar/2022 06:24:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2022 06:24:15] "GET /static/style.css HTTP/1.1" 404 -
127.0.0.1 - - [23/Mar/2022 06:24:15] "GET /favicon.ico HTTP/1.1" 404 -
